In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
from functools import partial

import os
import torch.nn.functional as F
from torch import nn

import emmental
from emmental import Meta
from emmental.data import EmmentalDataLoader, EmmentalDataset
from emmental.learner import EmmentalLearner
from emmental.model import EmmentalModel
from emmental.scorer import Scorer
from emmental.task import EmmentalTask
from modules.bert_module import BertModule
from modules.classification_module import ClassificationModule
from preprocessor import preprocessor
from task_config import LABEL_MAPPING

In [3]:
logger = logging.getLogger(__name__)

In [4]:
TASK_NAME = "MNLI"
DATA_DIR = os.environ["GLUEDATA"]
BERT_MODEL_NAME = "bert-large-uncased"
BATCH_SIZE = 32

# Initalize Emmental

In [5]:
emmental.init("logs/RTE_finetune",
    config={
        "meta_config": {"seed": 1},
        "model_config": {"device": 0, "dataparallel": True},
        "learner_config": {
            "n_epochs": 20,
            "valid_split": "val",
            "optimizer_config": {"optimizer": "adam", "lr": 1e-5},
            "lr_scheduler_config": {
                "lr_scheduler": "linear",  # "linear",
                "min_lr": 1e-7,
            },
        },
        "logging_config": {
            "counter_unit": "epoch",
            "evaluation_freq": 0.2,
            "checkpointing": True,
            "checkpointer_config": {
                "checkpoint_metric": {"RTE/SuperGLUE/val/accuracy":"max"},
                "checkpoint_freq": 1,
            },
        },
    }
)

[2019-05-31 07:20:26,566][INFO] emmental.meta:95 - Setting logging directory to: logs/RTE_finetune/2019_05_31/07_20_26
[2019-05-31 07:20:26,575][INFO] emmental.meta:56 - Loading Emmental default config from /home/hazymturk/vincent/emmental/src/emmental/emmental-default-config.yaml.
[2019-05-31 07:20:26,576][INFO] emmental.meta:143 - Updating Emmental config from user provided config.


# Build Emmental task

In [6]:
def ce_loss(task_name, immediate_ouput_dict, Y, active):
    module_name = f"{task_name}_pred_head"
    return F.cross_entropy(
        immediate_ouput_dict[module_name][0][active], (Y.view(-1) - 1)[active]
    )

In [7]:
def output(task_name, immediate_ouput_dict):
    module_name = f"{task_name}_pred_head"
    return F.softmax(immediate_ouput_dict[module_name][0], dim=1)

In [8]:
mtl_model = EmmentalModel(name="GLUE_single_task")

[2019-05-31 07:20:35,708][INFO] emmental.model:44 - Created emmental model GLUE_single_task that contains task set().
[2019-05-31 07:20:35,710][INFO] emmental.model:58 - Moving model to GPU (cuda:0).


In [9]:
mtl_model.load(
    "logs/RTE_finetune/2019_05_31/05_38_02/best_model_RTE_SuperGLUE_val_accuracy.pth"
)

[2019-05-31 07:20:42,908][INFO] emmental.model:412 - [GLUE_multi_task] Model loaded from logs/RTE_finetune/2019_05_31/05_38_02/best_model_RTE_SuperGLUE_val_accuracy.pth
[2019-05-31 07:20:42,910][INFO] emmental.model:58 - Moving model to GPU (cuda:0).


In [10]:
TASK_NAME = "RTE"

In [11]:
from superglue.parse_RTE import get_RTE_dataloaders

In [12]:
from superglue.task_config import SuperGLUE_LABEL_MAPPING, SuperGLUE_TASK_METRIC_MAPPING

In [13]:
BERT_OUTPUT_DIM = 768 if "base" in BERT_MODEL_NAME else 1024
TASK_CARDINALITY = (
    len(SuperGLUE_LABEL_MAPPING[TASK_NAME].keys())
    if SuperGLUE_LABEL_MAPPING[TASK_NAME] is not None
    else 1
)

emmental_task = EmmentalTask(
    name=TASK_NAME,
    module_pool=nn.ModuleDict(
        {
            "bert_module": BertModule(BERT_MODEL_NAME),
            f"{TASK_NAME}_pred_head": nn.Linear(BERT_OUTPUT_DIM, TASK_CARDINALITY),
        }
    ),
    task_flow=[
        {
            "name": "input",
            "module": "bert_module",
            "inputs": [("_input_", "token_ids"), ("_input_", "token_segments")],
        },
        {
            "name": f"{TASK_NAME}_pred_head",
            "module": f"{TASK_NAME}_pred_head",
            "inputs": [("input", 1)],
        },
    ],
    loss_func=partial(ce_loss, TASK_NAME),
    output_func=partial(output, TASK_NAME),
    scorer=Scorer(metrics=SuperGLUE_TASK_METRIC_MAPPING[TASK_NAME]),
)

[2019-05-31 07:20:43,886][INFO] pytorch_pretrained_bert.modeling:583 - loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased.tar.gz from cache at ./cache/214d4777e8e3eb234563136cd3a49f6bc34131de836848454373fa43f10adc5e.abfbb80ee795a608acbf35c7bf2d2d58574df3887cdd94b355fc67e03fddba05
[2019-05-31 07:20:43,888][INFO] pytorch_pretrained_bert.modeling:591 - extracting archive file ./cache/214d4777e8e3eb234563136cd3a49f6bc34131de836848454373fa43f10adc5e.abfbb80ee795a608acbf35c7bf2d2d58574df3887cdd94b355fc67e03fddba05 to temp dir /tmp/tmp62cwhxb9
[2019-05-31 07:20:54,448][INFO] pytorch_pretrained_bert.modeling:601 - Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "type_vocab_size": 2,
  "vocab_size":

In [14]:
mtl_model.remove_task("MNLI")

[2019-05-31 07:20:59,245][INFO] emmental.model:136 - Task (MNLI) not in the current model, skip...


In [15]:
mtl_model.add_task(emmental_task)

[2019-05-31 07:20:59,309][INFO] emmental.model:58 - Moving model to GPU (cuda:0).


In [16]:
DATA_DIR = os.environ["SUPERGLUEDATA"]
dataloaders = get_RTE_dataloaders(
    data_dir=DATA_DIR,
    task_name=TASK_NAME,
    splits=["train", "val"],
    max_sequence_length=200,
    max_data_samples=None,
    tokenizer_name=BERT_MODEL_NAME,
    batch_size=BATCH_SIZE,
)

[2019-05-31 07:20:59,335][INFO] superglue.tokenizer:8 - Loading Tokenizer bert-large-uncased
[2019-05-31 07:21:00,132][INFO] pytorch_pretrained_bert.tokenization:190 - loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-vocab.txt from cache at /home/hazymturk/.cache/torch/pytorch_pretrained_bert/9b3c03a36e83b13d5ba95ac965c9f9074a99e14340c523ab405703179e79fc46.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


/home/hazymturk/vincent/emmental-tutorials/superglue/data/RTE/train.jsonl
{'premise': 'No Weapons of Mass Destruction Found in Iraq Yet.', 'hypothesis': 'Weapons of Mass Destruction Found in Iraq.', 'label': 'not_entailment', 'idx': 0}


[2019-05-31 07:21:02,437][INFO] superglue.parse_RTE:123 - Loaded train for RTE.


/home/hazymturk/vincent/emmental-tutorials/superglue/data/RTE/val.jsonl
{'premise': 'Dana Reeve, the widow of the actor Christopher Reeve, has died of lung cancer at age 44, according to the Christopher Reeve Foundation.', 'hypothesis': 'Christopher Reeve had an accident.', 'label': 'not_entailment', 'idx': 0}


[2019-05-31 07:21:02,702][INFO] superglue.parse_RTE:123 - Loaded val for RTE.


In [17]:
mtl_model.score(dataloaders["val"])

{'RTE/SuperGLUE/val/accuracy': 0.8303249097472925}